# Find the stimuli that maximally differentiates LN model from the Convnet trained on natural scenes

In [1]:
import numpy as np
from os.path import expanduser
import os
import json
import theano
import pyret.filtertools as ft
import pyret.visualizations as pyviz
import deepretina.visualizations as viz
from deepretina.preprocessing import datagen, loadexpt
from keras.models import model_from_json
import h5py
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.optimize import minimize
from numpy.linalg import norm
%matplotlib inline
#%matplotlib qt

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

Using gpu device 0: GeForce GT 750M (CNMeM is disabled)


### Filenames and data directories

In [2]:
architecture_filename = 'architecture.json'
#whitenoise_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.salamander/2015-11-08 15.42.33 convnet/')
#whitenoise_weight_filename = 'epoch018_iter01300_weights.h5' # .63 cc on held-out
naturalscenes_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.salamander/2015-11-07 16.52.44 convnet/')
naturalscenes_weight_filename = 'epoch038_iter02700_weights.h5' # .53 cc on held-out
ln_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.nirum/2015-11-08 04.41.18 LN/')
ln_weight_filename = 'epoch010_iter00750_weights.h5' # .468 cc on held-out

## Load convnet trained on natural scenes

In [3]:
naturalscenes_architecture_data = open(naturalscenes_data_dir + architecture_filename, 'r')
naturalscenes_architecture_string = naturalscenes_architecture_data.read()
naturalscenes_model = model_from_json(naturalscenes_architecture_string)
naturalscenes_model.load_weights(naturalscenes_data_dir + naturalscenes_weight_filename)

## Load LN model

In [4]:
ln_architecture_data = open(ln_data_dir + architecture_filename, 'r')
ln_architecture_string = ln_architecture_data.read()
ln_model = model_from_json(ln_architecture_string)
ln_model.load_weights(ln_data_dir + ln_weight_filename)

# Define model separation and constraint

In [5]:
def model_separation(stimulus):
    stimulus = stimulus.reshape((1,40,50,50))
    ln_response = ln_model.predict(stimulus)[0][0]
    convnet_response = naturalscenes_model.predict(stimulus)[0][0]
    return -(convnet_response - ln_response)**2

In [6]:
constraint = {}
def unit_norm_constraint(stimulus):
    return (1 - norm(stimulus))**2
constraint['fun'] = unit_norm_constraint
constraint['type'] = 'eq'

# Unconstrained optimization

In [ ]:
initial_guess = np.random.randn(1*40*50*50)
res = minimize(model_separation, x0=initial_guess)

# Constrained optimization

In [7]:
initial_guess_two = np.random.randn(1*40*50*50)/300
res_constrained = minimize(model_separation, x0=initial_guess_two, constraints=constraint)

ValueError: GpuDownsampleFactorMax:Was not able to allocate output!
Apply node that caused the error: GpuDownsampleFactorMax{(2, 2),True}(GpuElemwise{Composite{(i0 * ((i1 + i2) + Abs((i1 + i2))))}}[(0, 1)].0)
Toposort index: 21
Inputs types: [CudaNdarrayType(float32, 4D)]
Inputs shapes: [(1, 8, 50, 50)]
Inputs strides: [(0, 3844, 62, 1)]
Inputs values: ['not shown']
Outputs clients: [[GpuReshape{2}(GpuDownsampleFactorMax{(2, 2),True}.0, MakeVector{dtype='int64'}.0)]]

Debugprint of the apply node: 
GpuDownsampleFactorMax{(2, 2),True} [@A] <CudaNdarrayType(float32, 4D)> ''   
 |GpuElemwise{Composite{(i0 * ((i1 + i2) + Abs((i1 + i2))))}}[(0, 1)] [@B] <CudaNdarrayType(float32, 4D)> ''   
   |CudaNdarrayConstant{[[[[ 2.08915925]]]]} [@C] <CudaNdarrayType(float32, (True, True, True, True))>
   |GpuSubtensor{::, ::, int64:int64:, int64:int64:} [@D] <CudaNdarrayType(float32, 4D)> ''   
   | |GpuCorrMM_gradInputs{valid, (1, 1)} [@E] <CudaNdarrayType(float32, 4D)> ''   
   | | |GpuContiguous [@